## Importing modules and packages

In [1]:
# Importing packages
import pandas as pd
import numpy  as np
import folium
import os.path
import shapely
from IPython.display import IFrame

# Importing modules
import gr_mapmatch # Contains functions that perform the map matching of roads
import gr_placematch # Contains functions that perform the map matching of places
import gr_utils # Contains useful geometry functions
import gr_plot # Contains plotting routines
import gr_process

/Users/svanwal/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Input parameters

In [2]:
trailname = 'gr16' # Name of the hiking trail to be considered (will search for trail.csv or trail.gpx as sources)

## Loading data

In [3]:
filename = 'data_output/' + trailname + '_processed.csv'
data = pd.read_csv(filename,dtype={'highway':str, 'surface': str, 'tracktype':str},index_col=0)
filename_gpx = 'data_input/' + trailname + '.gpx'
filename_csv = 'data_output/' + trailname + '.csv'
if not os.path.isfile(filename_csv): # The GPX file was not processed into a clean CSV file before
    if not os.path.isfile(filename_gpx): # The GPX file does not exist, throw error
        raise ValueError(f'The GPX file <{filename_gpx}> was not found! Please make sure it exists.')
    else: # The GPX file exists, so convert it into a clean CSV file
        print(f'Converting GPX file <{filename_gpx}> into cleaned CSV file <{filename_csv}>...')
        gr_utils.process_gpx(filename_gpx,filename_csv)
        print('Completed conversion.')
print(f'Loading trail points from <{filename_csv}>...')
trail = pd.read_csv(filename_csv) # Now read the cleaned CSV file into a DataFrame (latitude, longitude, elevation)
print('Finished loading.')

Loading trail points from <data_output/gr16.csv>...
Finished loading.


## Plotting

In [4]:
# Calculating focus point
focus = gr_plot.get_focus(trail)

In [5]:
# Plotting development type
# filepath = gr_plot.show_development(data,focus)
# IFrame(filepath, width=1000, height=500)

In [6]:
# Plotting traffic type
# filepath = gr_plot.show_traffic(data,focus)
# IFrame(filepath, width=1000, height=500)

In [7]:
# Plotting paved type
# filepath = gr_plot.show_paved(data,focus)
# IFrame(filepath, width=1000, height=500)

In [8]:
# Plotting paved type with highway/surface/tracktype labels
filepath = gr_plot.show_paved_detail(data,focus)
IFrame(filepath, width=1000, height=500)

In [9]:
# Plotting GR type with highway/surface/tracktype labels
filepath = gr_plot.show_type_detail(data,focus)
IFrame(filepath, width=1000, height=500)